<a href="https://colab.research.google.com/github/cedamusk/earthquake-prediction-using-Python/blob/main/STA_LTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install obspy #installs obspy

In [ ]:
import numpy as np
from obspy import read
from obspy.signal.trigger import classic_sta_lta, plot_trigger
import matplotlib.pyplot as plt

In [ ]:
!wget https://examples.obspy.org/ev0_6.a01.gse2 -O sample_data.gse2 #Download sample seismic data

In [ ]:
#read the seismic data
st=read("sample_data.gse2")
tr=st[0]#we'll use the first trace

In [ ]:
tr.filter('bandpass', freqmin=1, freqmax=20)#apply a bandpass filter

In [ ]:
#Calculate STA/LTA
sta_length=3#Short term average window (in seconds)
lta_length=30#Long term average window (in seconds)
cft=classic_sta_lta(tr.data, int(sta_length*tr.stats.sampling_rate), int(lta_length*tr.stats.sampling_rate))

In [ ]:
#Define trigger on and off thresholds
on_trigger=3.5
off_trigger=1.5

In [ ]:
#Plot the results
plt.figure(figsize=(12,8))
plot_trigger(tr, cft, on_trigger, off_trigger)
plt.show()

In [ ]:
#Print detected events
triggers=[]
trigger_on=False
for i, ratio in enumerate(cft):
  if not trigger_on and ratio>on_trigger:
    triggers.append(("on", tr.stats.starttime+i/tr.stats.sampling_rate))
    trigger_on=True
  elif trigger_on and ratio<off_trigger:
    triggers.append(("off", tr.stats.starttime+i/tr.stats.sampling_rate))
    trigger_on=False

In [ ]:
print("Detected events:")
for trigger in triggers:
  print(f"{'Trigger on' if trigger[0]=='on'else'Trigger off'}at{trigger[1]}")